In [1]:
import pandas as pd

df = pd.read_pickle('/home/er/Documents/CIRAD/colibri/data/merged_cleaned_pub/2023-10-4_13-57-55/data.pkl')
df.head(10)

,Platform,DOI,Title,Abstract,Keywords
0,WoS,10.1016/j.earscirev.2022.104214,How does soil water status influence the fate ...,Due to its influence on multiple soil processe...,Carbon sequestration; Organic matter dynamics;...
1,WoS,10.1016/j.jaridenv.2017.02.001,Variation in soil carbon stocks with depth alo...,Deep soil carbon (> 30 cm) is the primary pool...,Deep carbon; Toposequence; Mediterranean soils...
2,WoS,10.1016/j.soilbio.2018.06.014,Substrate identity and amount overwhelm temper...,The size of the soil carbon sink depends on th...,Soil organic carbon; Soil organic matter; Micr...
3,WoS,10.1186/s13750-021-00221-3,What is the impact of human wastewater biosoli...,"Background: Human wastewater biosolids, hereaf...",Land application; Meta-analysis; Sewage sludge...
4,WoS,10.1016/j.catena.2021.105227,Influence of land use on hydro-physical soil p...,The paramos biome of the northern Andes is a c...,Hydrological services; Soil hydrology; Edaphol...
5,WoS,10.1111/ejss.12492,The role of soil depth in the evaluation of ma...,The aim of the current survey was to determine...,NaN
6,WoS,10.1111/gcb.15489,"Long-term, amplified responses of soil organic...",Soil organic carbon (SOC) is the largest carbo...,carbon inputs; carbon sequestration; dissolved...
7,WoS,10.1016/j.catena.2023.107409,Digital mapping of soil organic carbon using r...,Soil organic carbon (SOC) has attracted a lot ...,Soil organic carbon; Environmental variables; ...
8,WoS,10.5194/soil-7-785-2021,Transformation of n-alkanes from plant to soil...,Despite the importance of soil organic matter ...,NaN
9,WoS,10.1111/nph.18458,A distinct sensitivity to the priming effect b...,Soil organic carbon (SOC) is a mixture of vari...,carbon availability; general priming effect; r...


In [ ]:
def classifier(weights_file, data_file):
    from transformers import DistilBertTokenizer, DistilBertModel
    import torch

    model_name = "distilbert-base-uncased"
    model = DistilBertModel.from_pretrained(model_name)
    tokenizer = DistilBertTokenizer.from_pretrained(model_name)
    tokenizer.add_tokens(
        ["SOC", "SOM", "meta-analysis", "metaanalysis", "meta-analyses", "metaanalyses"]
    )
    model.resize_token_embeddings(len(tokenizer))
    model.load_state_dict(torch.load(weights_file))
    model.eval()

    def preprocess_data(text_list, tokenizer, max_length):
        text = tokenizer(
            text_list,
            padding="max_length",
            truncation=True,
            max_length=max_length,
            return_tensors="pt",
        )
        attention_masks = text["attention_mask"]
        return text["input_ids"], attention_masks

    # Replace 'new_data_text_list' with your actual new data
    new_data_text_list = ["Sample text 1", "Sample text 2", ...]

    # Preprocess the new data
    new_data_input_ids, new_data_attention_masks = preprocess_data(
        new_data_text_list, tokenizer, max_length=config["padding_length"]
    )

    # Perform inference
    with torch.no_grad():
        # Move input to the same device as the model (GPU or CPU)
        new_data_input_ids = new_data_input_ids.to(device)
        new_data_attention_masks = new_data_attention_masks.to(device)

        # Forward pass through the model
        outputs = model(
            input_ids=new_data_input_ids, attention_mask=new_data_attention_masks
        )

        # Apply the classification head to get logits
        logits = classification_head(outputs.last_hidden_state[:, 0, :])

        # Convert logits to probabilities using softmax
        probabilities = torch.softmax(logits, dim=1)

        # Get predicted labels (0 or 1)
        predicted_labels = torch.argmax(probabilities, dim=1)

    # Convert predicted labels to a list
    predicted_labels_list = predicted_labels.tolist()

classifier("/home/er/Documents/Cirad/colibri/data/merged_cleaned_pub/2023-9-26_15-28-47/data.pkl")